### Setup

In [80]:
import pandas as pd
import ast
import os

In [81]:
local = pd.read_csv('datasets/crawler/local_codes.csv')
foreign = pd.read_csv('datasets/crawler/foreign_codes.csv')
codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
    .reset_index(drop=True)
)

### Methods

In [108]:
# Returns resuts table df
def getResultsTable(file):
    resultsParty = file.loc['resultsParty', 'currentResults']

    results = (
        pd.DataFrame(ast.literal_eval(resultsParty))
        .drop(['absoluteMajority', 'constituenctyCounter', 'imageKey', 'mandates', 'presidents'], axis=1)
        .rename(columns={'acronym': 'party'})
        .set_index('party')
    )

    return results

# Returns metadata df
def getMetadata(file, row):
    cr = pd.Series( # Current Results
        file['currentResults']
        .drop(['availableMandates', 'compensation', 'displayMessage', 'hasNoVoting', 'resultsParty'
            , 'tie', 'tieMessage', 'totalBoycotts', 'totalForeignBoycotts', 'totalLocalBoycotts'
            , 'totalMandates', 'totalParishesApproved'])
    )
    
    loc = pd.Series([row['parish'], row['county'], row['district'], row['territoryKey']], index=['parish', 'county', 'district', 'territoryKey'])

    metadata = (
        pd.concat([loc, cr])
        .to_frame()
        .T
        .set_index('territoryKey')
    )

    return metadata

# Returns results df
def getResults(file, metadata):
    r = getResultsTable(file)
    r['tkey'] = metadata.index[0]
    r['parish'] = metadata['territoryName'].iloc[0]

    r = r.reset_index().set_index(['tkey', 'party'])

    return r

# Creates directories
def mkdir():
        path = '/datasets/Results/' 

        working_dir = os.getcwd()
        newpath = working_dir + path

        if not os.path.exists(newpath):
            os.makedirs(newpath)

### Create Results files

In [109]:
results = pd.DataFrame()
metadata = pd.DataFrame()

for i, row in codes.iterrows():
    loc = row['territoryKey'].split('-')[0].capitalize()
    file = (
        pd.read_csv('datasets/crawler/' + loc + '/' + row['district'] + '/' + row['county'] + '/' + row['parish'] + '.csv')
        .set_index('index')
    )
    
    metadata = pd.concat([metadata, getMetadata(file, row)])
    # ! results broken
    # results = pd.concat([results, getResults(file, metadata)])

In [110]:
metadata

,parish,county,district,blankVotes,blankVotesPercentage,nullVotes,nullVotesPercentage,numberParishes,numberVoters,percentageVoters,subscribedVoters,totalVoters
territoryKey,,,,,,,,,,,,
LOCAL-430101,Altares,Angra do Heroísmo,Açores,13,2.8200000000000003,3,0.65,1,461,54.36,848,461
LOCAL-430102,Angra (Nossa Senhora da Conceição),Angra do Heroísmo,Açores,31,1.8,19,1.1,1,1720,46.24,3720,1720
LOCAL-430103,Angra (Santa Luzia),Angra do Heroísmo,Açores,33,2.73,14,1.16,1,1210,52.95,2285,1210
LOCAL-430104,Angra (São Pedro),Angra do Heroísmo,Açores,35,1.85,10,0.53,1,1887,56.96,3313,1887
LOCAL-430105,Angra (Sé),Angra do Heroísmo,Açores,7,1.1400000000000001,9,1.46,1,615,55.91,1100,615
...,...,...,...,...,...,...,...,...,...,...,...,...
FOREIGN-930299,Postos Consulares da China,China,Fora da Europa,128,2.08,1937,31.46,1,6157,12.57,48982,6157
FOREIGN-920799,Postos Consulares dos Estados Unidos da América,Estados Unidos da América,Fora da Europa,19,0.17,5592,51.26,1,10910,16.72,65242,10910
FOREIGN-919999,Postos Consulares dos países de África,Países de África,Fora da Europa,28,0.8,562,16.14,1,3483,5.48,63584,3483


In [ ]:
results

percentage  validVotesPercentage  votes  \
tkey         party                                                         
LOCAL-430101 PPD/PSD.CDS-PP.PPM       43.17                 44.72    199   
             PS                       34.27                 35.51    158   
             CH                       12.15                 12.58     56   
             B.E.                      2.17                  2.25     10   
             IL                        1.95                  2.02      9   
...                                     ...                   ...    ...   
             JPP                       0.22                  0.33     13   
             MPT.A                     0.17                  0.26     10   
             ND                        0.13                  0.21      8   
             R.I.R.                    0.12                  0.18      7   
             E                         0.08                  0.13      5   

                                  parish  
tkey         party                        
LOCAL-430101 PPD/PSD.CDS-PP.PPM  Altares  
             PS                  Altares  
             CH                  Altares  
             B.E.                Altares  
             IL                  Altares  
...                                  ...  
             JPP                 Altares  
             MPT.A               Altares  
             ND                  Altares  
             R.I.R.              Altares  
             E                   Altares  

[45736 rows x 4 columns]

In [ ]:
mkdir()

results.to_csv('datasets/Results/results.csv')
metadata.to_csv('datasets/Results/metadata.csv')

### Explore Data

In [ ]:
results = pd.read_csv('datasets/Results/results.csv')
metadata = pd.read_csv('datasets/Results/metadata.csv')

parties = results['party'].unique()
results = results.set_index(['parish', 'party'])

idx = pd.IndexSlice

In [ ]:
results.loc['Angra (Sé)', 'PS']

C:\Users\druid\AppData\Local\Temp\ipykernel_5312\3641195549.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  results.loc['Angra (Sé)', 'PS']


,,tkey,percentage,validVotesPercentage,votes
parish,party,,,,
Angra (Sé),PS,LOCAL-430105,24.88,25.54,153


In [ ]:
# Results by Party
# for i, party in parties:
#     results.loc[idx[:, party], idx[:]]

## Database

In [ ]:
# results
# metadata


In [ ]:
# Send to SQL
import mysql.connector
from dotenv import load_dotenv

load_dotenv()
user = os.getenv('USER')
password = os.getenv('PW')

cnx = mysql.connector.connect(
    user=user,
    password=password,
    host="localhost",
    database="legislativas"
)


